In [16]:
import requests
from bs4 import BeautifulSoup
import tarfile
import io
import os
from pathlib import Path

In [2]:
APACHE_SPAM_ASSASSIN = "https://spamassassin.apache.org/old/publiccorpus"
response = requests.get(APACHE_SPAM_ASSASSIN)
response.raise_for_status()

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')
file_paths = [link.get('href') for link in soup.find_all('a')]
file_paths = [path for path in file_paths if path.split('.')[-1] == 'bz2']
file_paths

['20021010_easy_ham.tar.bz2',
 '20021010_hard_ham.tar.bz2',
 '20021010_spam.tar.bz2',
 '20030228_easy_ham.tar.bz2',
 '20030228_easy_ham_2.tar.bz2',
 '20030228_hard_ham.tar.bz2',
 '20030228_spam.tar.bz2',
 '20030228_spam_2.tar.bz2',
 '20050311_spam_2.tar.bz2']

In [20]:
for file_path in file_paths:
    response = requests.get(f"{APACHE_SPAM_ASSASSIN}/{file_path}")
    response.raise_for_status()

    file_object = io.BytesIO(response.content)
    tar = tarfile.open(fileobj=file_object, mode="r:bz2")

    extract_dir = Path('../data')
    extract_path = extract_dir.joinpath(Path(file_path))

    extract_dir.mkdir(exist_ok=True)
    tar.extractall(extract_path)

    tar.close()

In [21]:
def get_directory_size(directory):
    total = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total += os.path.getsize(fp)

    return total

size = get_directory_size("../data")
print(f"Directory size: {size} bytes")

Directory size: 60298644 bytes
